<a href="https://colab.research.google.com/github/mukthakaja/microsoft-phi-3-slm-support-agent/blob/main/phi3_fine_tune_bnb_n4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate wandb
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, set_seed, BitsAndBytesConfig

set_seed(47) # for reproducibility

model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type = "nf4"
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config = bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "### Human: Can you tell about Configuration Steps for Okta Integration with Prism Central. ### Assistant:"

inputs = tokenizer(text, return_tensors = "pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens = 250, do_sample = False)

print(tokenizer.decode(outputs[0], skip_special_tokens = False))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

quantizing using the bitsandbytes library

In [ ]:
!pip install -qqq -upgrade transformers bitsandbytes accelerate datasets


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [ ]:
!pip -q install huggingface_hub transformers peft bitsandbytes
!pip -q install trl xformers
!pip -q install datasets
!pip install torch>=1.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
import torch

from huggingface_hub import ModelCard, ModelCardData, HfApi

from datasets import load_dataset

from trl import SFTTrainer
import yaml

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"
finetuned_model_name = "finetuned_phi3_model" # after finetuning process

In [ ]:
dataset = "rghosh8/supportGPT_data"

split = "train"
max_seq_length = 1024
num_train_epochs = 1
license = "apache-2.0"
learning_rate = 1.41e-5
per_device_batch_train_rate = 2
gradient = 1


In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, trust_remote_code=True)



from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

target_modules = ["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"]

peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    target_modules = target_modules,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model.add_adapter(peft_config)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.add_special_tokens({'pad_token' : '<PAD'})

split_dataset = load_dataset(dataset, split="train[:1%]")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

In [ ]:

def format(samples) :
  return tokenizer(samples['text'], padding = "max_length", truncation = True)

In [ ]:
# need to define training parameters
from transformers import TrainingArguments
hf_username = "mukthakaja"

training_args = TrainingArguments(
   evaluation_strategy = "steps",
   output_dir = finetuned_model_name,
   per_device_train_batch_size = 1,
   per_device_eval_batch_size = 1,
   learning_rate = 2e-5,
   max_steps = 100,
   num_train_epochs = 1,
   save_strategy = "epoch",
   gradient_accumulation_steps = 16,
   gradient_checkpointing = True,
   logging_steps = 10,
   optim = "paged_adamw_32bit",
   lr_scheduler_type="linear"

)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
   model = model,
   args = training_args,
   train_dataset = split_dataset,
   eval_dataset = eval_dataset,
   dataset_text_field = "text",
   tokenizer = tokenizer,
   max_seq_length = 64,
   formatting_func = format
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

In [ ]:
eval_dataset = load_dataset(dataset, split = "train[-1%:]")

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,0.000000,2.522361
20,0.000000,2.536473
30,0.000000,2.536835
40,0.000000,2.537692
50,0.000000,2.539504
60,0.000000,2.535894
70,0.000000,2.535864
80,0.000000,2.539123
90,0.000000,2.538960
100,0.000000,2.537945


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:399: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed 

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = quantization_config
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
text = "### Human: Can you tell about All user VMs will not be powered on automatically after power outage in a 2 node Nutanix AHV cluster. ### Assistant:"

inputs = tokenizer(text, return_tensors = "pt").to(0)
outputs = model.generate(inputs.input_ids, max_new_tokens = 250, do_sample = False)

print(tokenizer.decode(outputs[0], skip_special_tokens = False))

### Human: Can you tell about All user VMs will not be powered on automatically after power outage in a 2 node Nutanix AHV cluster. ### Assistant: In a Nutanix AHV (Acropolis Hypervisor) cluster, VMs (Virtual Machines) are managed by the Nutanix software, which provides a unified platform for managing both compute and storage resources.

When a power outage occurs, the behavior of VMs in an AHV cluster depends on the configuration and the state of the VMs at the time of the outage. Here are some points to consider:

1. **VM State**: If a VM was running at the time of the power outage, its state will be saved to the host's memory. When power is restored, the VM will be in a "stopped" state, and you will need to manually start it again.

2. **VM Configuration**: VMs can be configured to start automatically after a power outage. This is typically done by setting the "Startup" option in the VM's settings to "Automatic." When this option is enabled, the VM will attempt to start automaticall